In [40]:
import pandas as pd
from pandas import DataFrame
import dateutil.parser
import json


view_party=pd.read_csv('../refdata/view_party.csv').drop(['cmp','euprofiler','ees','castles_mair','huber_inglehart','ray','benoit_laver','chess'],axis=1)
partyColumns=view_party.columns

view_election=pd.read_csv('../refdata/view_election.csv')
electionColumns=view_election.columns

view_election=view_election.merge(view_party,on='party_id',how='left',suffixes=('','_2'))
view_cabinet=pd.read_csv('../refdata/view_cabinet.csv')

view_election=view_election.drop([x for x in view_election.columns if x.endswith('_2')],axis=1)
view_election.election_date=view_election.election_date.apply(dateutil.parser.parse)

NationalElections=view_election[view_election.election_type=='parliament']
EuroElections=view_election[view_election.election_type=='ep']


def getLastElection(t):
    return t[t.election_date==t.election_date.max()]

NationalElections=NationalElections.groupby('country_name_short').apply(getLastElection).reset_index(drop=True)
EuroElections=EuroElections.groupby('country_name_short').apply(getLastElection).reset_index(drop=True)

NationalElections.to_csv('../refdata/national_elections.csv',index=False)
EuroElections.to_csv('../refdata/euro_elections.csv',index=False)


def getPartyJson(code):
    topLevelFields=['country_id','country_name_short','country_name','election_id','election_date','seats_total']
    
    nat = NationalElections[code==NationalElections.country_name_short] \
        .sort_values(['seats','vote_share','party_name_short'],ascending=False)
    r=nat[topLevelFields].reset_index(drop=True).ix[0].to_dict()
    r['election_date']=str(r['election_date'])
    r.update({'parties' : [x[1][nat.columns.difference(topLevelFields)].dropna().to_dict() for x in nat.iterrows()]})

    return r


data={code:getPartyJson(code) for code in NationalElections['country_name_short'].drop_duplicates() if not code in ['AUS','CAN','ISR','NZL']}

with open('../public/parlgov.json', 'w') as outfile:
    json.dump(data, outfile,indent=4, default='')